In [1]:
import pandas as pd
import numpy as np
import cbbpy.mens_scraper as s

from sklearn.preprocessing import MinMaxScaler

In [2]:
# game_info_df, boxscore_df, pbp_df = s.get_games_season(2024)

In [47]:
try:
    game_info_df = pd.read_csv('raw_data/game_info_df.csv')
    boxscore_df = pd.read_csv('raw_data/boxscore_df.csv')
    pbp_df = pd.read_csv('raw_data/pbp_df.csv')
except:
    game_info_df, boxscore_df, pbp_df = s.get_games_season(2024)
    game_info_df.to_csv('game_info_df.csv',index=False)
    boxscore_df.to_csv('boxscore_df.csv',index=False)
    pbp_df.to_csv('pbp_df.csv',index=False)


## Summary

Assumptions:
* Refs, networks, arena capacity, date, time of day and tournament do not impact score

## Games

### Filter

In [4]:
game_info_df.shape

(5117, 27)

In [5]:
game_info_df.head()

,game_id,home_team,home_id,home_rank,home_record,home_score,away_team,away_id,away_rank,away_record,...,game_day,game_time,game_loc,arena,arena_capacity,attendance,tv_network,referee_1,referee_2,referee_3
0,401575451,Kansas Jayhawks,2305,1.0,1-0,99,North Carolina Central Eagles,2428,NaN,0-1,...,"November 06, 2023",05:00 PM PST,"Lawrence, KS",Allen Fieldhouse,0,16300,BIG12|ESPN+,Gerry Pollard,Amy Bonner,Chance Moore
1,401576147,Duke Blue Devils,150,2.0,1-0,92,Dartmouth Big Green,159,NaN,0-1,...,"November 06, 2023",06:00 PM PST,"Durham, NC",Cameron Indoor Stadium,0,9314,ACCN,Jamie Luckie,Ryan Sassano,Clare Aubry
2,401577640,Purdue Boilermakers,2509,3.0,1-0,98,Samford Bulldogs,2535,NaN,0-1,...,"November 06, 2023",03:30 PM PST,"West Lafayette, IN",Mackey Arena,0,14876,BTN,D.J. Carstensen,Edwin Young,Jourdan Love
3,401581835,Michigan State Spartans,127,4.0,0-1,76,James Madison Dukes,256,NaN,1-0,...,"November 06, 2023",05:30 PM PST,"East Lansing, MI",Breslin Center,0,14797,BTN,Brian Dorsey,Chad Barlow,Brian McNutt
4,401577598,Marquette Golden Eagles,269,5.0,1-0,92,Northern Illinois Huskies,2459,NaN,0-1,...,"November 06, 2023",05:30 PM PST,"Milwaukee, WI",Fiserv Forum,0,16352,NaN,Lamar Simpson,Peter Larson,Ed Corliss


In [6]:
game_info_df.columns

Index(['game_id', 'home_team', 'home_id', 'home_rank', 'home_record',
       'home_score', 'away_team', 'away_id', 'away_rank', 'away_record',
       'away_score', 'home_win', 'num_ots', 'is_conference', 'is_neutral',
       'is_postseason', 'tournament', 'game_day', 'game_time', 'game_loc',
       'arena', 'arena_capacity', 'attendance', 'tv_network', 'referee_1',
       'referee_2', 'referee_3'],
      dtype='object')

In [7]:
print('Shape before filtering:', game_info_df.shape)
filtered_df = game_info_df[game_info_df['is_postseason']==False]
# filtered_df = filtered_df[(game_info_df['home_record']!='') & (game_info_df['away_record']!='')]
filtered_df = filtered_df.dropna(subset=['home_record','away_record'])
print('Shape after filtering:', filtered_df.shape)

Shape before filtering: (5117, 27)
Shape after filtering: (4607, 27)


In [8]:
exclude_cols = ['home_win','home_score', 'away_score', 'home_id', 'away_id','num_ots', 'is_postseason', 'tournament', 'game_time', 'game_loc',
       'arena', 'arena_capacity', 'attendance', 'tv_network', 'referee_1',
       'referee_2', 'referee_3']
       
date_col = 'game_day'
onehot_cols = ['home_team','away_team']
bool_cols = ['is_conference','is_neutral']
rank_cols = ['home_rank','away_rank']
records_cols = ['home_record','away_record']
# target_cols = ['home_score','away_score']


### Transform

In [9]:
# Exclude Cols
transformed_df = filtered_df.drop(exclude_cols, axis=1)

# Date col
transformed_df['game_day'] = pd.to_datetime(transformed_df['game_day'])

# One Hot Encode
# transformed_df = pd.get_dummies(transformed_df, columns=onehot_cols, prefix = ['home','away'], dtype=int)

# Convert Bool to Int
for col in bool_cols:
    transformed_df[col] = transformed_df[col].astype(int)

# Replace NULL Ranks with 99
for col in rank_cols:
    transformed_df[col] = transformed_df[col].fillna(99)

# Conver Record cols to Wins and Losses cols
for col in records_cols:
    team_type = col.split('_')[0]
    transformed_df[col] = transformed_df[col].astype(str)
    wins = transformed_df.loc[:,col].apply(lambda x: x.split('-')[0])
    losses = transformed_df.loc[:,col].apply(lambda x: x.split('-')[1])    
    transformed_df[team_type+'Team_wins'] = wins
    transformed_df[team_type+'Team_losses'] = losses
    transformed_df = transformed_df.drop(col,axis=1)

print(transformed_df.shape)

(4607, 12)


### Transformed Preview

In [10]:
transformed_df.head()

,game_id,home_team,home_rank,away_team,away_rank,is_conference,is_neutral,game_day,homeTeam_wins,homeTeam_losses,awayTeam_wins,awayTeam_losses
0,401575451,Kansas Jayhawks,1.0,North Carolina Central Eagles,99.0,0,0,2023-11-06,1,0,0,1
1,401576147,Duke Blue Devils,2.0,Dartmouth Big Green,99.0,0,0,2023-11-06,1,0,0,1
2,401577640,Purdue Boilermakers,3.0,Samford Bulldogs,99.0,0,0,2023-11-06,1,0,0,1
3,401581835,Michigan State Spartans,4.0,James Madison Dukes,99.0,0,0,2023-11-06,0,1,1,0
4,401577598,Marquette Golden Eagles,5.0,Northern Illinois Huskies,99.0,0,0,2023-11-06,1,0,0,1


## Boxscores

In [11]:
#TODO: convert home and away teams to vectors in transformed df?
#TODO; convert boxscores to vectors
    # TODO: find top 7-8 players by min played and find avg stats using those players and not whole team

In [48]:
boxscore_df.columns

Index(['game_id', 'team', 'player', 'player_id', 'position', 'starter', 'min',
       'fgm', 'fga', '2pm', '2pa', '3pm', '3pa', 'ftm', 'fta', 'oreb', 'dreb',
       'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts'],
      dtype='object')

In [49]:
stat_cols = ['fgm', 'fga', '2pm', '2pa', '3pm', '3pa', 'ftm', 'fta', 'oreb', 'dreb',
       'reb', 'ast', 'stl', 'blk', 'to', 'pf']
exclude_boxscore_cols = ['player', 'player_id', 'position', 'starter', 'min']


### Filter to Boxscore Totals

In [50]:
boxscores_filtered = boxscore_df[(boxscore_df.player_id == 'TOTAL')]

### Normalize Boxscore Stats

In [51]:
# Normalize Stats
scaler = MinMaxScaler()
boxscores_filtered[stat_cols] = scaler.fit_transform(boxscores_filtered[stat_cols])

/var/folders/z_/b2p80qvn353dbq0c1vfq0q600000gq/T/ipykernel_80236/2050784139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxscores_filtered[stat_cols] = scaler.fit_transform(boxscores_filtered[stat_cols])


In [52]:
boxscores_filtered.head()

,game_id,team,player,player_id,position,starter,min,fgm,fga,2pm,...,fta,oreb,dreb,reb,ast,stl,blk,to,pf,pts
12,401575451,North Carolina Central Eagles,TEAM,TOTAL,TOTAL,False,NaN,0.363636,0.558824,0.285714,...,0.352941,0.272727,0.250000,0.2875,0.189189,0.304348,0.055556,0.257143,0.325,56
25,401575451,Kansas Jayhawks,TEAM,TOTAL,TOTAL,False,NaN,0.709091,0.549020,0.530612,...,0.196078,0.121212,0.589286,0.4625,0.918919,0.173913,0.166667,0.285714,0.325,99
41,401576147,Dartmouth Big Green,TEAM,TOTAL,TOTAL,False,NaN,0.345455,0.549020,0.306122,...,0.333333,0.303030,0.339286,0.3625,0.297297,0.217391,0.055556,0.371429,0.400,54
54,401576147,Duke Blue Devils,TEAM,TOTAL,TOTAL,False,NaN,0.690909,0.578431,0.632653,...,0.235294,0.151515,0.500000,0.4125,0.486486,0.260870,0.333333,0.257143,0.450,92
68,401577640,Samford Bulldogs,TEAM,TOTAL,TOTAL,False,NaN,0.309091,0.666667,0.244898,...,0.274510,0.424242,0.339286,0.4125,0.297297,0.478261,0.000000,0.428571,0.475,45


### Convert Stat Columns into One stats_vector

In [53]:
# Get top 8 guys by minutes. Convert their stats into matrices. For input when predicting, will need to find averages and fill in each value
# OR just get their totals?
# boxscores_filtered['stats_vector'] = list(np.array(boxscores_filtered.loc[:,stat_cols]))
# bs_v = boxscores_filtered.drop(stat_cols+exclude_boxscore_cols,axis=1).reset_index(drop=True)
# bs_v
# test_boxscore.assign(new=pd.factorize(df.))

# Ignore vectors for now... 
bs_v = boxscores_filtered.drop(exclude_boxscore_cols,axis=1).reset_index(drop=True)
bs=bs_v.copy()

In [64]:
bs.shape

(10234, 19)

In [39]:
# df_rolling  = df_rolling.groupby(['team'], group_keys=False).apply(find_team_averages)
def find_team_averages(team):
    rolling = team.rolling(5).mean()
    return rolling

In [68]:
bs_grouped = bs.groupby(['team'], group_keys=False)[stat_cols].apply(find_team_averages)
rolling_cols = [f'{col}_5' for col in bs_grouped.columns]
bs_grouped.columns = rolling_cols

In [74]:
include_cols = ['game_id','team','pts']+rolling_cols
bs_grouped_with_game_info = pd.concat([bs_v,bs_grouped],axis=1)[include_cols]
bs_grouped_with_game_info.head()

,game_id,team,pts,fgm_5,fga_5,2pm_5,2pa_5,3pm_5,3pa_5,ftm_5,fta_5,oreb_5,dreb_5,reb_5,ast_5,stl_5,blk_5,to_5,pf_5
0,401575451,North Carolina Central Eagles,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,401575451,Kansas Jayhawks,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,401576147,Dartmouth Big Green,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,401576147,Duke Blue Devils,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,401577640,Samford Bulldogs,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# Join in transformed on game ID and the teams


home_boxscores_df = transformed_df.rename(columns={'home_team':'team'})
home_boxscores_df['home_team']=home_boxscores_df['team']

away_boxscores_df = transformed_df.rename(columns={'away_team':'team'})
away_boxscores_df['away_team']=away_boxscores_df['team']

home_boxscores = pd.merge(bs_grouped_with_game_info, home_boxscores_df, how='left', on=['game_id','team'], validate='one_to_one').dropna()
# home_boxscores = home_boxscores.drop(['team'],axis=1)
away_boxscores = pd.merge(bs_grouped_with_game_info, away_boxscores_df, how='left', on=['game_id','team'], validate='one_to_one').dropna()[include_cols]
# combined_boxscores = pd.merge(home_boxscores,away_boxscores,on='game_id',validate='one_to_one',suffixes=('_home','_away'))
combined_boxscores = pd.merge(home_boxscores,away_boxscores,on='game_id',validate='one_to_one',suffixes=('_home','_away'))
combined_boxscores

,game_id,team_home,pts_home,fgm_5_home,fga_5_home,2pm_5_home,2pa_5_home,3pm_5_home,3pa_5_home,ftm_5_home,...,ftm_5_away,fta_5_away,oreb_5_away,dreb_5_away,reb_5_away,ast_5_away,stl_5_away,blk_5_away,to_5_away,pf_5_away
0,401606133,South Alabama Jaguars,82,0.469091,0.600000,0.379592,0.485333,0.327273,0.476923,0.375,...,0.540,0.576471,0.442424,0.478571,0.5175,0.356757,0.191304,0.255556,0.240000,0.405
1,401581813,Omaha Mavericks,70,0.461818,0.572549,0.387755,0.496000,0.290909,0.407692,0.410,...,0.315,0.337255,0.278788,0.407143,0.4000,0.237838,0.295652,0.144444,0.411429,0.500
2,401577562,Presbyterian Blue Hose,66,0.494545,0.535294,0.428571,0.512000,0.281818,0.311538,0.415,...,0.365,0.388235,0.248485,0.464286,0.4275,0.378378,0.460870,0.177778,0.320000,0.450
3,401594151,Air Force Falcons,80,0.421818,0.486275,0.302041,0.368000,0.381818,0.423077,0.380,...,0.400,0.423529,0.321212,0.492857,0.4775,0.389189,0.234783,0.066667,0.314286,0.475
4,401604346,Nicholls Colonels,51,0.476364,0.609804,0.334694,0.456000,0.445455,0.538462,0.380,...,0.335,0.372549,0.345455,0.446429,0.4550,0.308108,0.243478,0.255556,0.291429,0.380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,401577676,Columbia Lions,64,0.490909,0.572549,0.383673,0.488000,0.372727,0.419231,0.280,...,0.240,0.258824,0.327273,0.407143,0.4200,0.372973,0.208696,0.188889,0.348571,0.425
3929,401587749,Milwaukee Panthers,80,0.472727,0.639216,0.338776,0.453333,0.427273,0.600000,0.310,...,0.530,0.521569,0.490909,0.471429,0.5325,0.372973,0.243478,0.244444,0.274286,0.425
3930,401596915,Fordham Rams,79,0.418182,0.547059,0.302041,0.402667,0.372727,0.492308,0.290,...,0.330,0.325490,0.230303,0.442857,0.4050,0.405405,0.330435,0.244444,0.360000,0.425
3931,401594303,Quinnipiac Bobcats,81,0.523636,0.615686,0.387755,0.528000,0.445455,0.446154,0.305,...,0.290,0.329412,0.290909,0.389286,0.3925,0.345946,0.373913,0.100000,0.268571,0.335


In [81]:
combined_boxscores.loc[combined_boxscores['game_id']==401606133,['fgm_5_home','fgm_5_away','away_team','team_away','home_team','team_home','pts_home','pts_away']]

,fgm_5_home,fgm_5_away,away_team,team_away,home_team,team_home,pts_home,pts_away
0,0.469091,0.494545,Denver Pioneers,Denver Pioneers,South Alabama Jaguars,South Alabama Jaguars,82,75


In [82]:
combined_boxscores.columns

Index(['game_id', 'team_home', 'pts_home', 'fgm_5_home', 'fga_5_home',
       '2pm_5_home', '2pa_5_home', '3pm_5_home', '3pa_5_home', 'ftm_5_home',
       'fta_5_home', 'oreb_5_home', 'dreb_5_home', 'reb_5_home', 'ast_5_home',
       'stl_5_home', 'blk_5_home', 'to_5_home', 'pf_5_home', 'home_rank',
       'away_team', 'away_rank', 'is_conference', 'is_neutral', 'game_day',
       'homeTeam_wins', 'homeTeam_losses', 'awayTeam_wins', 'awayTeam_losses',
       'home_team', 'team_away', 'pts_away', 'fgm_5_away', 'fga_5_away',
       '2pm_5_away', '2pa_5_away', '3pm_5_away', '3pa_5_away', 'ftm_5_away',
       'fta_5_away', 'oreb_5_away', 'dreb_5_away', 'reb_5_away', 'ast_5_away',
       'stl_5_away', 'blk_5_away', 'to_5_away', 'pf_5_away'],
      dtype='object')

In [83]:
# new_bs = combined_boxscores.copy()
team_cols = ['away_team','team_away','home_team','team_home','pts_home','pts_away']
home_rows = []
away_rows = []
for i, row in combined_boxscores.iterrows():    
    row['is_home']=1
    row['team']=row['home_team']
    row['score']=row['pts_home']
    home_rows.append(row.copy())

    row['is_home']=0
    row['team']=row['away_team']
    row['score']=row['pts_away']
    away_rows.append(row.copy())

final_df = pd.concat([pd.DataFrame(home_rows), pd.DataFrame(away_rows)])
    
        
    


In [85]:
final_df = final_df.drop(team_cols,axis=1)

In [86]:
final_df.columns

Index(['game_id', 'fgm_5_home', 'fga_5_home', '2pm_5_home', '2pa_5_home',
       '3pm_5_home', '3pa_5_home', 'ftm_5_home', 'fta_5_home', 'oreb_5_home',
       'dreb_5_home', 'reb_5_home', 'ast_5_home', 'stl_5_home', 'blk_5_home',
       'to_5_home', 'pf_5_home', 'home_rank', 'away_rank', 'is_conference',
       'is_neutral', 'game_day', 'homeTeam_wins', 'homeTeam_losses',
       'awayTeam_wins', 'awayTeam_losses', 'fgm_5_away', 'fga_5_away',
       '2pm_5_away', '2pa_5_away', '3pm_5_away', '3pa_5_away', 'ftm_5_away',
       'fta_5_away', 'oreb_5_away', 'dreb_5_away', 'reb_5_away', 'ast_5_away',
       'stl_5_away', 'blk_5_away', 'to_5_away', 'pf_5_away', 'is_home', 'team',
       'score'],
      dtype='object')

In [88]:
final_df.loc[final_df['game_id']==401606133,:].to_csv('tetser.csv')

### Save Data

In [89]:
final_df.to_csv('processed_data/data.csv',index=False)